Scrapes the novelupdates webpage, http://www.novelupdates.com/
for novel information and cleans it.
Finally saves the dataset as a csv file.

Didn't find any easy way to get all the novel ids, hence, the novels listing is first used to get all ids.

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novels_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="

In [76]:
# Get the maximum number of novel pages from the listings

def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novels_page + '1')
novels_max_pages = get_novel_list_max_pages(page)

In [77]:
# Get all novel ids from the novels listing
def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novels_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids, columns=['Id'])
df.to_csv("novel_ids_small.csv", index=False)      # temporary

In [75]:
df = pd.read_csv("novel_ids_small.csv") # temporary

def parse_novel_page(id):
    # open page
    # parse for wanted elements (create a map for these? name-> how to find it)
    # return these as a map (name -> result)
    # add these as new columns to the df
    return id + 3
    
df['test'] = df.apply(lambda x: parse_novel_page(x['Id']), axis=1)
df.head

<bound method NDFrame.head of        Id   test
0    1173   1176
1    1175   1178
2    1177   1180
3   13038  13041
4    1171   1174
5    2042   2045
6    6612   6615
7   11133  11136
8    9343   9346
9    8233   8236
10   7959   7962
11   2696   2699
12  13155  13158
13   2742   2745
14   7145   7148
15   5400   5403
16   9314   9317
17  10927  10930
18   1179   1182
19  12408  12411
20  13080  13083
21   1069   1072
22  13197  13200
23  12584  12587
24   1181   1184
25   1183   1186
26   3992   3995
27   1185   1188
28   8674   8677
29  10925  10928
..    ...    ...
45  11499  11502
46   8512   8515
47   7756   7759
48    399    402
49   2151   2154
50   8074   8077
51  13558  13561
52   7151   7154
53    564    567
54   4300   4303
55   9038   9041
56   2085   2088
57  11393  11396
58  11677  11680
59  13384  13387
60   2693   2696
61  13576  13579
62   2765   2768
63   9120   9123
64   7591   7594
65  13207  13210
66  10394  10397
67   2568   2571
68   2366   2369
69   2771   2774
7